In [ ]:
# パッケージをインストール
%pip install -qe ..

In [2]:
import duckdb
import japanize_matplotlib  # noqa: F401
import matplotlib.dates as mdates
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

## 基本設定


In [3]:
plt.style.use("ggplot")

In [ ]:
with duckdb.connect("../data/raw/etl_from_sf.duckdb", read_only=True) as con:
    df = con.execute("SELECT * FROM raw_clean_with_pool").df()
df.head()

In [ ]:
df.info()

In [ ]:
df.columns

In [7]:
raw_clean_with_pool = df

In [ ]:
print(raw_clean_with_pool.shape)
print(raw_clean_with_pool.dtypes)

In [ ]:
print(raw_clean_with_pool.isna().sum())
print(raw_clean_with_pool.astype(str).duplicated().sum())

## データ変換


In [10]:
# タイムスタンプを日時に変換
raw_clean_with_pool["datetime"] = pd.to_datetime(raw_clean_with_pool["hour_ts"], unit="s")

In [11]:
# IDからプールアドレスとインデックスを抽出
raw_clean_with_pool["pool_address"] = raw_clean_with_pool["id"].str.split("-").str[0]
raw_clean_with_pool["block_index"] = raw_clean_with_pool["id"].str.split("-").str[1]

In [ ]:
# 重複確認と除去
print(f"重複行数: {raw_clean_with_pool.duplicated().sum()}")
raw_clean_with_pool = raw_clean_with_pool.drop_duplicates()
print(f"重複除去後の行数: {raw_clean_with_pool.shape[0]}")

In [13]:
# フィーティアごとのプール数
fee_tier_counts = raw_clean_with_pool["fee_tier"].value_counts().reset_index()
fee_tier_counts.columns = ["fee_tier", "count"]

In [ ]:
# 基本統計量の確認
raw_clean_with_pool.describe()

In [15]:
# 数値カラムのキャスト
num_cols = [
    "volume_usd",
    "tvl_usd",
    "fees_usd",
    "open_price",
    "high_price",
    "low_price",
    "close_price",
    "liquidity",
    "volume_token0",
    "volume_token1",
    "tx_count",
    "tick",
    "sqrt_price",
]
for c in num_cols:
    raw_clean_with_pool[c] = pd.to_numeric(raw_clean_with_pool[c], errors="coerce")

### 異常値の検出 & クリーニング


In [16]:
#  clip で下限 0、 or 上限 percentile 99.9% に制限
raw_clean_with_pool["tvl_usd"] = raw_clean_with_pool["tvl_usd"].clip(lower=0)
upper = raw_clean_with_pool["volume_usd"].quantile(0.999)
raw_clean_with_pool = raw_clean_with_pool[raw_clean_with_pool["volume_usd"] <= upper]

In [ ]:
# 欠損の確認（型変換で NaN が入る可能性があるため）
print(raw_clean_with_pool[num_cols].isna().sum())
raw_clean_with_pool = raw_clean_with_pool.dropna(subset=num_cols)

## 可視化


### 分布の可視化：Plotly Express を使ったヒストグラム＆箱ひげ図


In [ ]:
# 各カラムに対してヒストグラムと箱ひげ図を作成
for c in ["volume_usd", "tvl_usd", "fees_usd"]:
    # ヒストグラム作成
    fig, ax = plt.subplots(figsize=(10, 6))
    ax.hist(raw_clean_with_pool[c], bins=100, alpha=0.75)
    ax.set_title(f"{c} の分布 (logスケール)")
    ax.set_xlabel(c)
    ax.set_ylabel("件数")
    ax.set_yscale("log")  # y軸をログスケールに設定
    plt.tight_layout()
    plt.show()

    # 箱ひげ図作成
    fig, ax = plt.subplots(figsize=(10, 6))
    boxplot = ax.boxplot(
        raw_clean_with_pool[c], showfliers=True, flierprops={"marker": "o", "markerfacecolor": "red", "markersize": 5}
    )
    ax.set_title(f"{c} の箱ひげ図（外れ値のみ）")
    ax.set_ylabel(c)
    ax.set_xticks([])  # x軸の目盛りを非表示（単一の箱ひげ図では不要）
    plt.tight_layout()
    plt.show()

### 時系列パターンの可視化：日次／主要プールごと


In [ ]:
# 日次集計プロット
daily = (
    raw_clean_with_pool.assign(date=lambda df: df["datetime"].dt.date).groupby("date")["volume_usd"].sum().reset_index()
)

plt.figure(figsize=(12, 6))
sns.lineplot(data=daily, x="date", y="volume_usd", marker="o", markersize=4, linewidth=1.5)
plt.title("日次 Volume USD の推移")
plt.xlabel("日付")
plt.ylabel("Volume USD")
# 日付フォーマットの調整
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter("%Y-%m-%d"))
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# トップ5プールの推移
top5 = raw_clean_with_pool.groupby("pool_address")["volume_usd"].sum().nlargest(5).index

# オプション1: 個別プロット（元のコードと同様）
for p in top5:
    dfp = raw_clean_with_pool.query("pool_address == @p")
    dfp_agg = dfp.set_index("datetime").resample("D")["volume_usd"].sum().reset_index()

    plt.figure(figsize=(12, 6))
    sns.lineplot(data=dfp_agg, x="datetime", y="volume_usd", marker="o", markersize=4, linewidth=1.5)
    plt.title(f"プール {p} の時間推移")
    plt.xlabel("日時")
    plt.ylabel("Volume USD")
    # 日付フォーマットの調整
    plt.gca().xaxis.set_major_formatter(mdates.DateFormatter("%Y-%m-%d"))
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

In [ ]:
# 数値変数の分布を Seaborn で可視化
numeric_cols = ["volume_usd", "tvl_usd", "fees_usd", "tx_count", "token0_price", "token1_price", "liquidity"]

for col in numeric_cols:
    # ヒストグラム
    plt.figure(figsize=(10, 6))
    sns.histplot(data=raw_clean_with_pool, x=col, bins=50, kde=True, alpha=0.75)
    plt.title(f"Distribution of {col}")
    plt.xlabel(col)
    plt.ylabel("Frequency")
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()

    # 箱ひげ図
    plt.figure(figsize=(8, 6))
    sns.boxplot(y=raw_clean_with_pool[col], color="lightblue")
    plt.title(f"Boxplot of {col}")
    plt.ylabel(col)
    plt.grid(True, axis="y", alpha=0.3)
    plt.tight_layout()
    plt.show()

    # 対数変換した分布も確認 (歪みが大きい場合に有用)
    log_data = np.log1p(raw_clean_with_pool[col])
    plt.figure(figsize=(10, 6))
    sns.histplot(x=log_data, bins=50, kde=True, alpha=0.75)
    plt.title(f"Distribution of log1p({col})")
    plt.xlabel(f"log1p({col})")
    plt.ylabel("Frequency")
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()

    # バイオリンプロット (分布の形状をより詳細に確認)
    plt.figure(figsize=(8, 6))
    sns.violinplot(y=raw_clean_with_pool[col], cut=0)
    plt.title(f"Violin plot of {col}")
    plt.ylabel(col)
    plt.grid(True, axis="y", alpha=0.3)
    plt.tight_layout()
    plt.show()

### 4. 多変量相関の可視化：散布行列（ペアプロット）


In [ ]:
# 特徴量を絞って散布行列
cols_corr = ["volume_usd", "tvl_usd", "fees_usd", "tx_count"]
sampled_data = raw_clean_with_pool.sample(5000)  # サンプル数を制限

# Seabornでペアプロットを作成
g = sns.pairplot(sampled_data[cols_corr], height=2.5, aspect=1)

# 対角線上の要素を非表示にする
for i in range(len(cols_corr)):
    g.axes[i, i].set_visible(False)

# タイトル設定
plt.suptitle("主要指標のペアプロット", fontsize=16)
plt.subplots_adjust(top=0.95)  # タイトル用のスペースを調整
plt.show()

### 異常検知関連特徴量の探索的作成


In [ ]:
# ボリュームの変化率 (前Hour比)
raw_clean_with_pool["volume_change_1h"] = raw_clean_with_pool.groupby("pool_address")["volume_usd"].pct_change()

# TVLとボリュームの比率
raw_clean_with_pool["tvl_volume_ratio"] = raw_clean_with_pool["tvl_usd"] / (
    raw_clean_with_pool["volume_usd"] + 1e-9
)  # ゼロ除算を防ぐ

# 新しい特徴量の分布を可視化
new_features = ["volume_change_1h", "tvl_volume_ratio"]

for col in new_features:
    # 外れ値を除外したデータを取得（可視化用）
    # 特に tvl_volume_ratio は非常に大きな値を持つ可能性があるため
    data = raw_clean_with_pool[col].copy()
    q1, q3 = np.percentile(data.dropna(), [25, 75])
    iqr = q3 - q1
    upper_bound = q3 + 5 * iqr  # 5 * IQRをカットオフとして使用

    # ヒストグラム
    fig, ax = plt.subplots(figsize=(10, 6))
    ax.hist(data.clip(upper=upper_bound), bins=50, alpha=0.75)
    ax.set_title(f"Distribution of {col}")
    ax.set_xlabel(col)
    ax.set_ylabel("Frequency")
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()

    # 箱ひげ図
    fig, ax = plt.subplots(figsize=(8, 6))
    ax.boxplot(
        data,
        vert=True,
        patch_artist=True,
        boxprops=dict(facecolor="lightblue", color="blue"),
        flierprops=dict(marker="o", markerfacecolor="red", markersize=5),
    )
    ax.set_title(f"Boxplot of {col}")
    ax.set_ylabel(col)
    ax.set_xticks([])  # x軸目盛りを非表示
    plt.grid(True, axis="y", alpha=0.3)
    plt.tight_layout()
    plt.show()

In [ ]:
from datetime import datetime

import pandas as pd

# 日付を適切に変換
start_date = pd.to_datetime("2025-04-16").date()
end_date = pd.to_datetime("2025-05-05").date()

# 日付のみの欠損をチェック
if isinstance(daily["date"].iloc[0], datetime):
    daily_dates = set(d.date() for d in daily["date"])
else:
    daily_dates = set(daily["date"])

# 期間内のすべての日付を生成
all_dates = pd.date_range(start=start_date, end=end_date).date
missing_dates = [d for d in all_dates if d not in daily_dates]

print("欠損している日付:")
for d in missing_dates:
    print(f"{d}")

# 元データから時間単位の欠損をチェック
print("\n時間単位の詳細:")

# datetimeの一覧を取得（元データから）
if "datetime" in raw_clean_with_pool.columns:
    # ユニークな日時を取得
    unique_timestamps = raw_clean_with_pool["datetime"].sort_values().unique()

    # 期間内のみをフィルタリング
    period_timestamps = [ts for ts in unique_timestamps if start_date <= ts.date() <= end_date]

    # 日付ごとの時間をまとめる
    date_hours = {}
    for ts in period_timestamps:
        date = ts.date()
        hour = ts.hour
        if date not in date_hours:
            date_hours[date] = []
        date_hours[date].append(hour)

    # 各日付の存在する時間を表示
    for date in sorted(date_hours.keys()):
        hours = sorted(date_hours[date])
        missing_hours = [h for h in range(24) if h not in hours]
        print(f"{date}: データあり: {hours}, 欠損: {missing_hours}")

    # データがまったくない日付を表示
    complete_missing_dates = [d for d in missing_dates if d not in date_hours]
    if complete_missing_dates:
        print("\n完全に欠損している日付:")
        for d in complete_missing_dates:
            print(f"{d}: すべての時間帯でデータなし")
else:
    print("元データに datetime 列がありません")